In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
from vani.analyzer import Analyzer

In [3]:
# Initialize analyzer
cluster_settings = dict(
    dashboard_port=3446,
    local_directory="/var/tmp/dask",
    log_file="digio.worker.log"
)
vn = Analyzer(debug=True, cluster_settings=cluster_settings)

# Analysis configuration
log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: tcp://192.168.66.200:43033
distributed.scheduler - INFO -   dashboard at:       192.168.66.200:3446
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:40219'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:43275'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:33887'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:34147'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:37685'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:39451'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:36449'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.66.200:36691'
distribute

Dashboard link: http://192.168.66.200:3446/status
Dashboard link: http://192.168.66.200:3447/status
#!/usr/bin/env bash

#BSUB -J dask-worker
#BSUB -G asccasc
#BSUB -q pdebug
#BSUB -W 120

/usr/workspace/iopp/projects/digio-venv-lassen/bin/python -m distributed.cli.dask_worker tcp://192.168.66.200:38957 --nthreads 1 --nprocs 16 --memory-limit 93.13GiB --name dummy-name --nanny --death-timeout 7200 --local-directory /var/tmp/dask/tmid

Dashboard link: http://192.168.66.200:3448/status
#!/usr/bin/env bash

#BSUB -J dask-worker
#BSUB -G asccasc
#BSUB -q pdebug
#BSUB -W 120

/usr/workspace/iopp/projects/digio-venv-lassen/bin/python -m distributed.cli.dask_worker tcp://192.168.66.200:40695 --nthreads 1 --nprocs 16 --memory-limit 93.13GiB --name dummy-name --nanny --death-timeout 7200 --local-directory /var/tmp/dask/proc_id

Dashboard link: http://192.168.66.200:3449/status
#!/usr/bin/env bash

#BSUB -J dask-worker
#BSUB -G asccasc
#BSUB -q pdebug
#BSUB -W 120

/usr/workspace/iopp/projects/d

In [4]:
#analysis = vn.analyze_parquet_logs(log_dir, depth=10, persist_stats=True, stats_file_prefix="cm1_")

In [5]:
#files = vn.ensure_logs_partitioned(log_dir=log_dir)

In [6]:
def wait_until_workers_alive(filter_group_index: str, n_workers: int = None):
    # Get current number of workers
    current_n_workers = len(vn.clients[filter_group_index].scheduler_info()['workers'])
    expected_n_workers = int(vn.n_workers_per_node if n_workers is None else n_workers)
    # Wait until enough number of workers alive
    while vn.clients[filter_group_index].status == 'running' and current_n_workers < expected_n_workers:
        # Log status
        vn.logger.debug(
            format_log(filter_group_index, f"{current_n_workers}/{expected_n_workers} workers running"))
        # Ensure loop
        vn.ensure_asyncio_loop()
        # Try correcting state
        # noinspection PyProtectedMember
        vn.clusters[filter_group_index]._correct_state()
        # Sleep a little
        sleep(1)
        # Get current number of workers
        current_n_workers = len(vn.clients[filter_group_index].scheduler_info()['workers'])
    # Print result
    vn.logger.debug(format_log(filter_group_index, "All workers alive"))

In [7]:
import dask
import dask.dataframe as dd
import functools
import glob
import json
import logging
import math
import os
import psutil
import socket
from anytree import PostOrderIter
from concurrent.futures import ThreadPoolExecutor, as_completed
from dask.distributed import Client, LocalCluster, get_task_stream, progress, wait
from distributed.diagnostics.plugin import SchedulerPlugin
from dask_jobqueue import LSFCluster
from time import perf_counter, sleep
from tqdm.auto import tqdm
from typing import Union
def compute_min_max(log_dir: str, json_key: str, filter_group_index: str):
    with open(f"{log_dir}/global.json") as file:
        global_metrics = json.load(file)
        min_val, max_val = global_metrics[json_key][0], global_metrics[json_key][1]
        return min_val, max_val

In [8]:
from vani.utils.logger import ElapsedTimeLogger, create_logger, format_log
filter_group_index='tmid'
wait_until_workers_alive(filter_group_index, vn.n_workers_per_node / 2)
#wait_workers = vn.clients[filter_group_index].submit()
#wait_workers.result()

In [9]:
import numpy as np
PARTITION_FOLDER = "partitioned_hari"
def indexed_ddf(log_dir: str, filter_group_index: str):
    ddf = dd.read_parquet(f"{log_dir}/*.parquet", index=False)
    #ddf[filter_group_index] = ddf[filter_group_index].astype(np.int64)
    ddf = ddf.set_index([filter_group_index])
    ddf = ddf.persist()
    #result = wait(ddf)
    return ddf
def merge(x, y):
    return {
        'read': {
            'uniq_ranks': np.union1d(x['read']['uniq_ranks'], y['read']['uniq_ranks']),
            'agg_dur': x['read']['agg_dur'] + y['read']['agg_dur'],
            'total_io_size': x['read']['total_io_size'] + y['read']['total_io_size'],
            'uniq_filenames': np.union1d(x['read']['uniq_filenames'], y['read']['uniq_filenames']),
            'bw_sum': x['read']['bw_sum'] + y['read']['bw_sum'],
            'ops': x['read']['ops'] + y['read']['ops'],
        },
        'write': {
            'uniq_ranks': np.union1d(x['write']['uniq_ranks'], y['write']['uniq_ranks']),
            'agg_dur': x['write']['agg_dur'] + y['write']['agg_dur'],
            'total_io_size': x['write']['total_io_size'] + y['write']['total_io_size'],
            'uniq_filenames': np.union1d(x['write']['uniq_filenames'], y['write']['uniq_filenames']),
            'bw_sum': x['write']['bw_sum'] + y['write']['bw_sum'],
            'ops': x['write']['ops'] + y['write']['ops'],
        },
        'metadata': {
            'uniq_ranks': np.union1d(x['metadata']['uniq_ranks'], y['metadata']['uniq_ranks']),
            'agg_dur': x['metadata']['agg_dur'] + y['metadata']['agg_dur'],
            'uniq_filenames': np.union1d(x['metadata']['uniq_filenames'], y['metadata']['uniq_filenames']),
            'ops': x['metadata']['ops'] + y['metadata']['ops'],
        }
    }
def compute_metrics(ddf, filter_group_index: str, interval):
    min_val, max_val = interval
    def filter(target_ddf, filter_group_index: str):
        # Select dataframes
        read_ddf = target_ddf[(target_ddf['io_cat'] == 1)]
        write_ddf = target_ddf[(target_ddf['io_cat'] == 2)]
        metadata_ddf = target_ddf[(target_ddf['io_cat'] == 3)]
        # TODO: on same compute run for all ddf.
        # Create tasks
        read_tasks = [
            read_ddf.index.unique() if filter_group_index == 'proc_id' else read_ddf['proc_id'].unique(),
            read_ddf['duration'].sum(),
            read_ddf['size'].sum(),
            read_ddf.index.unique() if filter_group_index == 'file_id' else read_ddf['file_id'].unique(),
            read_ddf['bandwidth'].sum(),
            read_ddf['index'].count(),
        ]
        write_tasks = [
            write_ddf.index.unique() if filter_group_index == 'proc_id' else write_ddf['proc_id'].unique(),
            write_ddf['duration'].sum(),
            write_ddf['size'].sum(),
            write_ddf.index.unique() if filter_group_index == 'file_id' else write_ddf['file_id'].unique(),
            write_ddf['bandwidth'].sum(),
            write_ddf['index'].count(),
        ]
        metadata_tasks = [
            metadata_ddf.index.unique() if filter_group_index == 'proc_id' else metadata_ddf['proc_id'].unique(),
            metadata_ddf['duration'].sum(),
            metadata_ddf.index.unique() if filter_group_index == 'file_id' else metadata_ddf['file_id'].unique(),
            metadata_ddf['index'].count(),
        ]
        filter_tasks = []
        filter_tasks.extend(read_tasks)
        filter_tasks.extend(write_tasks)
        filter_tasks.extend(metadata_tasks)
        # Compute all
        filter_results = dask.compute(*filter_tasks)
        # Clear dataframes
        del read_ddf
        del write_ddf
        del metadata_ddf
        del target_ddf
        # Arrange results
        read_start, read_end = 0, len(read_tasks)
        write_start, write_end = len(read_tasks), len(read_tasks) + len(write_tasks)
        metadata_start, metadata_end = len(read_tasks) + len(write_tasks), 0
        filter_result = {
            'read': {
                'uniq_ranks': filter_results[:read_end][0],
                'agg_dur': filter_results[:read_end][1],
                'total_io_size': filter_results[:read_end][2],
                'uniq_filenames': filter_results[:read_end][3],
                'bw_sum': filter_results[:read_end][4],
                'ops': filter_results[:read_end][5],
            },
            'write': {
                'uniq_ranks': filter_results[write_start:write_end][0],
                'agg_dur': filter_results[write_start:write_end][1],
                'total_io_size': filter_results[write_start:write_end][2],
                'uniq_filenames': filter_results[write_start:write_end][3],
                'bw_sum': filter_results[write_start:write_end][4],
                'ops': filter_results[write_start:write_end][5],
            },
            'metadata': {
                'uniq_ranks': filter_results[metadata_start:][0],
                'agg_dur': filter_results[metadata_start:][1],
                'uniq_filenames': filter_results[metadata_start:][2],
                'ops': filter_results[metadata_start:][3],
            }
        }
        # Return results
        return filter_result

    
    def get_empty():
        return {
            'read': {
                'uniq_ranks': [],
                'agg_dur': 0.0,
                'total_io_size': 0,
                'uniq_filenames': [],
                'bw_sum': 0.0,
                'ops': 0,
            },
            'write': {
                'uniq_ranks': [],
                'agg_dur': 0.0,
                'total_io_size': 0,
                'uniq_filenames': [],
                'bw_sum': 0.0,
                'ops': 0,
            },
            'metadata': {
                'uniq_ranks': [],
                'agg_dur': 0.0,
                'uniq_filenames': [],
                'ops': 0,
            }
        }
    def cal_len(x):
        return {
            'read': {
                'uniq_ranks': len(x['read']['uniq_ranks']),
                'agg_dur': x['read']['agg_dur'],
                'total_io_size': x['read']['total_io_size'],
                'uniq_filenames': len(x['read']['uniq_filenames']),
                'bw_sum': x['read']['bw_sum'],
                'ops': x['read']['ops'],
            },
            'write': {
                'uniq_ranks': len(x['write']['uniq_ranks']),
                'agg_dur': x['write']['agg_dur'],
                'total_io_size': x['write']['total_io_size'],
                'uniq_filenames': len(x['write']['uniq_filenames']),
                'bw_sum': x['write']['bw_sum'],
                'ops': x['write']['ops'],
            },
            'metadata': {
                'uniq_ranks': len(x['metadata']['uniq_ranks']),
                'agg_dur': x['metadata']['agg_dur'],
                'uniq_filenames': len(x['metadata']['uniq_filenames']),
                'ops': x['metadata']['ops']
            }
        }

    depth = 10
    next_tasks = 2 ** depth
    interval = math.ceil((max_val - min_val) * 1.0 / next_tasks)
    iterations = list(range(0, depth + 1))
    iterations.reverse()
    all_tasks = [0] * (depth + 1)
    time_range = range(min_val, max_val, interval)
    for i in iterations:
        tasks = []
        if i == depth:
            for start in time_range:
                stop = start + interval - 1
                target_ddf = ddf.loc[start:stop]
                tasks.append(dask.delayed(filter)(target_ddf, filter_group_index))
        else:
            next_tasks = len(all_tasks[i + 1])
            if next_tasks % 2 == 1:
                next_tasks = next_tasks - 1
            for t in range(0, next_tasks, 2):
                tasks.append(dask.delayed(merge)(all_tasks[i + 1][t], all_tasks[i + 1][t + 1]))
            next_tasks = len(all_tasks[i + 1])
            if next_tasks % 2 == 1:
                tasks.append(all_tasks[i + 1][next_tasks - 1])
            # TODO why are we calling len on everything?
            for t, next_tasks in enumerate(all_tasks[i + 1]):
                all_tasks[i + 1][t] = dask.delayed(cal_len)(next_tasks)
        all_tasks[i] = tasks
    for t, next_tasks in enumerate(all_tasks[0]):
        all_tasks[0][t] = dask.delayed(cal_len)(next_tasks)
    metrics = dask.compute(all_tasks)    
    #result = wait(metrics_futures)
    #metrics = client.gather(metrics_futures)
    return metrics

In [10]:
# json_index = 'tmid'
# futures = []
# args = [log_dir, json_index, filter_group_index]
# interval_ft = vn.clients[filter_group_index].submit(compute_min_max, *args, key=f'{filter_group_index}_compute_min_max')
# futures.append(interval_ft)
# args = [log_dir, filter_group_index]
# indexed_ft = vn.clients[filter_group_index].submit(indexed_ddf, *args, key=f'{filter_group_index}_indexed_ddf')
# futures.append(indexed_ft)
# args = [indexed_ft, filter_group_index, interval_ft]
# metrics_ft = vn.clients[filter_group_index].submit(compute_metrics, *args, key=f'{filter_group_index}_compute_metrics')
# futures.append(metrics_ft)

In [11]:
# from dask.distributed import as_completed
# from tqdm.auto import tqdm
# metrics_map = {}
# for future in tqdm(as_completed(futures)):
#     if "compute_metrics" in future.key:
#         metrics_map[future.key] = future.result(timeout=(2*60*60))
#     print(f'{future.key} completed')

In [12]:
# metrics_map

In [13]:
json_keys = dict(tmid='tmid', proc_id='proc_id', file_id='file_id')
futures = []
for filter_group_index in vn.filter_group_indices:
    json_index = json_keys[filter_group_index]
    args = [log_dir, json_index, filter_group_index]
    interval_ft = vn.clients[filter_group_index].submit(compute_min_max, *args, key=f'{filter_group_index}_compute_min_max')
    futures.append(interval_ft)
    args = [log_dir, filter_group_index]
    indexed_ft = vn.clients[filter_group_index].submit(indexed_ddf, *args, key=f'{filter_group_index}_indexed_ddf')
    futures.append(indexed_ft)
    args = [indexed_ft, filter_group_index, interval_ft]
    metrics_ft = vn.clients[filter_group_index].submit(compute_metrics, *args, key=f'{filter_group_index}_compute_metrics')
    futures.append(metrics_ft)


In [14]:
%%time
from dask.distributed import as_completed
from tqdm.auto import tqdm
metrics_map = {}
for future in tqdm(as_completed(futures)):
    if "compute_metrics" in future.key:
        metrics_map[future.key] = future.result(timeout=(2*60*60))
    print(f'{future.key} completed')

0it [00:00, ?it/s]

tmid_compute_min_max completed
proc_id_compute_min_max completed
file_id_compute_min_max completed
tmid_indexed_ddf completed
proc_id_indexed_ddf completed
file_id_indexed_ddf completed
proc_id_compute_metrics completed
tmid_compute_metrics completed


KilledWorker: ('finalize-1664667b-d4fc-486f-b8f9-a28a9b43b045', <WorkerState 'tcp://192.168.64.30:44433', name: LSFCluster-0-10, status: closed, memory: 0, processing: 1>)

In [15]:
# min_val, max_val = metrics_map["tmid_compute_min_max"]
# depth = 10
# next_tasks = 2 ** depth
# interval = math.ceil((max_val - min_val) * 1.0 / next_tasks)
# time_range = range(min_val, max_val, interval)
# for start in time_range:
#     print(start, start + interval - 1)

In [19]:
def merge_result(x, y):
    return {
        'read': {
            'uniq_ranks': x['read']['uniq_ranks'] + y['read']['uniq_ranks'],
            'agg_dur': x['read']['agg_dur'] + y['read']['agg_dur'],
            'total_io_size': x['read']['total_io_size'] + y['read']['total_io_size'],
            'uniq_filenames': x['read']['uniq_filenames'] + y['read']['uniq_filenames'],
            'bw_sum': x['read']['bw_sum'] + y['read']['bw_sum'],
            'ops': x['read']['ops'] + y['read']['ops'],
        },
        'write': {
            'uniq_ranks': x['write']['uniq_ranks'] + y['write']['uniq_ranks'],
            'agg_dur': x['write']['agg_dur'] + y['write']['agg_dur'],
            'total_io_size': x['write']['total_io_size'] + y['write']['total_io_size'],
            'uniq_filenames': x['write']['uniq_filenames'] + y['write']['uniq_filenames'],
            'bw_sum': x['write']['bw_sum'] + y['write']['bw_sum'],
            'ops': x['write']['ops'] + y['write']['ops'],
        },
        'metadata': {
            'uniq_ranks': x['metadata']['uniq_ranks'] + y['metadata']['uniq_ranks'],
            'agg_dur': x['metadata']['agg_dur'] + y['metadata']['agg_dur'],
            'uniq_filenames': x['metadata']['uniq_filenames'] + y['metadata']['uniq_filenames'],
            'ops': x['metadata']['ops'] + y['metadata']['ops'],
        }
    }

In [20]:
metrics_map['tmid_compute_metrics'][0][0]

[{'read': {'uniq_ranks': 1764,
   'agg_dur': 49288.523,
   'total_io_size': 78001370679164,
   'uniq_filenames': 10549344,
   'bw_sum': 6142663000000.0,
   'ops': 589699346},
  'write': {'uniq_ranks': 1764,
   'agg_dur': 3487.186,
   'total_io_size': 112322741322,
   'uniq_filenames': 10445412,
   'bw_sum': 565536830.0,
   'ops': 10730937},
  'metadata': {'uniq_ranks': 3990,
   'agg_dur': 80646.44,
   'uniq_filenames': 21260259,
   'ops': 114795803}}]

In [21]:
merge_result(metrics_map['proc_id_compute_metrics'][0][0][0],metrics_map['proc_id_compute_metrics'][0][0][1])

{'read': {'uniq_ranks': 1764,
  'agg_dur': 49288.535,
  'total_io_size': 78001370679164,
  'uniq_filenames': 10549348,
  'bw_sum': 6142664000000.0,
  'ops': 589699346},
 'write': {'uniq_ranks': 1764,
  'agg_dur': 3487.186,
  'total_io_size': 112322741322,
  'uniq_filenames': 10445412,
  'bw_sum': 565536830.0,
  'ops': 10730937},
 'metadata': {'uniq_ranks': 3990,
  'agg_dur': 80646.44,
  'uniq_filenames': 21260266,
  'ops': 114795803}}

In [22]:
metrics_map['file_id_compute_metrics'][0][0]

KeyError: 'file_id_compute_metrics'

In [ ]:
metrics_map

In [ ]:
metrics_map['file_id_indexed_ddf'].compute()

In [ ]:
metrics_map['tmid_compute_metrics'][0][10]

In [ ]:
metrics_map['proc_id_compute_metrics'][0][0]

In [ ]:
metrics_map['file_id_compute_metrics'][0][0][0]

In [ ]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
local_client = Client(LocalCluster(n_workers=16))

In [ ]:
local_client

In [ ]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet", index=False)

In [ ]:
values = dd.compute(ddf['tmid'].min(), ddf['tmid'].max(), ddf['file_id'].min(), ddf['file_id'].max(), ddf['proc_id'].min(), ddf['proc_id'].max())

In [ ]:
values

In [ ]:
min_val, max_val = -9223371986035283781, 9223369538921024184

next_tasks = 2 ** 10
interval = math.ceil((max_val - min_val) * 1.0 / next_tasks)
time_range = range(min_val, max_val, interval)
print(time_range)
for start in time_range:
    print(start, start + interval -1)